# Preprocessing

Convert the raw data from FormKeep (csv) and GetForm (xls) services
and save them into `raw-data-processed.csv`.

FormKeep files have been slightly modified manually to use
the same columns as GetForm files.

In [46]:
import glob
import logging
import numpy as np
import os
import pandas as pd
from collections import OrderedDict
from pandas import Series, DataFrame
import xml.etree.ElementTree as ET

# excel (xls, xlsx) reading also requires
# pip install xlrd
# pip install openpyxl

In [47]:
# RE patterns for files to load from
LOAD_FROM = ["raw_data/devanagari/*.csv",
             "raw_data/devanagari/*.xls",
             "raw_data/devanagari/*.xlsx",
             "raw_data/cyrillic/*.csv",
             "raw_data/cyrillic/*.xls",
             "raw_data/cyrillic/*.xlsx",
             "raw_data/latin_and_cyrillic/*.csv",
             "raw_data/latin_and_cyrillic/*.xls",
             "raw_data/latin_and_cyrillic/*.xlsx",
             ]
# folder to save the CSVs to
SAVE_TO = "csv/"

logging.basicConfig(level=logging.INFO, format='%(message)s')

In [78]:
# ----------------------------------------------------------------------
# Language normalisation

PUNCTUATION = ["(", "+", ")", "/", "&", " ", ".", ";", " - ", u"।"]

ANDS = [u"and", u"ий", u"и", u"і", u"й", u"та", u"and", u"&", u"और", u"व", u"तथा", "aur"]

LANGS_CORRECTIONS_SEARCH = OrderedDict([
    (u"most of cyrillic based.", ""),
    (u"(can read english more fluently than hindi)", ""),
    (u"my mobile set has only one language, hence such replies", ""),
    (u"थोड़ी", ""),  # a little
    (u"थोड़ी", ""),  # a little
    (u"(no script for kashmiri)", ""),
    (u"less fluently", ""),
    (u"некоторые славянские", ""),
    (u"most of latin-based", ""),
    (u"(i.e. scandinavian)", ""),
    (u"но весьма прилично", ""),
    (u"; не идеально", ""),
    (u"will see", ""),
    (u"языках", ""),  # languages
    (u"язык", ""),  # a language
    (u"на ", ""),  # on
    (u"basic", ""),
    (u"none", ""),
    (u"भाषा", ""),  # language
    (u"लिपीके", ""),  # script
    (u"सब", ""),  # all
    (u"на своем есть жи", "russian"),  # on its own?

    # correction for misaligned data
    (u"1", ""),
    (u"2", ""),
    (u"3", ""),
    (u"4", ""),

    (u"first language is russian but i'm more fluent in english", "russian,english"),
    (u"serbian (latin and cyrillic)", "serbian"),
    (u"chinese (simplified)", "chinese-simplified"),
    (u"chinese (mandarin)", "chinese-mandarin"),
    (u"spanish (mexican)", "spanish-mexican"),
    (u"scottish gaelic", "scottish-gaelic"),
    (u"(latin, greek?)", "latin,greek"),
    (u"classical greek", "greek-classical"),
    (u"serbo-croatian", "serbian,croatian"),
    (u"português br", "portuguese-brasilian"),
    (u"swiss german", "german-swiss"),
    (u"swissgerman", "german-swiss"),
    (u"german swiss", "german-swiss"),
    (u"uk english", "english"),
    (u"englush", "english"),
    (u"французский", "french"),
    (u"francais", "french"),
    (u"français", "french"),
    (u"belorussian", "belarusian"),
    (u"белорусском", "belarusian"),
    (u"белорусский", "belarusian"),
    (u"итальянский", "italian"),
    (u"netherlands", "dutch"),
    (u"нидерландский", "dutch"),
    (u"slovenština", "slovak"),
    (u"белоруссий", "belarusian"),
    (u"болгарский", "bulgarian"),
    (u"укр", "ukrainian"),
    (u"украинская", "ukrainian"),
    (u"латинский", "latin"),
    (u"англиский", "english"),
    (u"российский", "russian"),
    (u"кацапській", "russian"),
    (u"русс", "russian"),
    (u"русски", "russian"),
    (u"руссский", "russian"),
    (u"руском", "russian"),
    (u"украинском", "ukrainian"),
    (u"лацiнка", "latin"),
    (u"итальянсктй", "italian"),
    (u"русским", "russian"),
    (u"italiano", "italian"),
    (u"аеглийский", "english"),
    (u"руссе", "russian"),
    (u"українській", "ukrainian"),
    (u"исландский", "icelandic"),
    (u"украинский", "ukrainian"),
    (u"українська", "ukrainian"),
    (u"український", "ukrainian"),
    (u"ураинский", "ukrainian"),
    (u"англійська", "english"),
    (u"венгерский", "hungarian"),
    (u"angličtina", "english"),
    (u"английский", "english"),
    (u"англійський", "english"),
    (u"беларуский", "belarusian"),
    (u"белоруский", "belarusian"),
    (u"беларускі", "belarusian"),
    (u"білоруська", "belarusian"),
    (u"английском", "english"),
    (u"укаринский", "ukrainian"),
    (u"български", "bulgarian"),
    (u"латышский", "latvian"),
    (u"узбекский", "uzbek"),
    (u"norwegiam", "norwegian"),
    (u"російська", "russian"),
    (u"português", "portuguese"),
    (u"cantonese", "chinese-cantonese"),
    (u"slovensky", "slovak"),
    (u"эстонский", "estonian"),
    (u"slovakian", "slovak"),
    (u"казахский", "kazakh"),
    (u"корейский", "korean"),
    (u"испанский", "spanish"),
    (u"японский", "japanese"),
    (u"сербский", "serbian"),
    (u"rosyjski", "russian"),
    (u"englisch", "english"),
    (u"espanish", "spanish"),
    (u"english", "english"),
    (u"englich", "english"),
    (u"русском", "russian"),
    (u"російський", "russian"),
    (u"русскій", "russian"),
    (u"frysian", "frisian"),
    (u"русский", "russian"),
    (u"seedish", "swedish"),
    (u"deutsch", "german"),
    (u"немецкий", "german"),
    (u"німецький", "german"),
    (u"чешский", "czech"),
    (u"čeština", "czech"),
    (u"español", "spanish"),
    (u"enflish", "english"),
    (u"poland", "polish"),
    (u"польский", "polish"),
    (u"польском", "polish"),
    (u"польський", "polish"),
    (u"польська", "polish"),
    (u"eglish", "english"),
    (u"polski", "polish"),
    (u"inglés", "english"),
    (u"руский", "russian"),
    (u"česky", "czech"),
    (u"मराठी", "marathi"),
    (u"иврит", "hebrew"),
    (u"hindu", "hindi"),
    (u"český", "czech"),
    (u"dutcg", "dutch"),
    (u"anglais", "english"),
    (u"अँग्रेजी", "english"),
    (u"अंग्रेज़ी", "english"),
    (u"अंग्रेजी", "english"),
    (u"अंग्रेज़ी", "english"),
    (u"इंग्रजी", "english"),
    (u"इंग्लिश", "english"),
    (u"उर्दू", "urdu"),
    (u"गुजराती", "gujarati"),
    (u"गढ़वाली", "garhwali"),
    (u"नेपाली", "nepali"),
    (u"मलयालम", "malayalam"),
    (u"मैथिली", "maithili"),
    (u"संस्कृत", "sanskrit"),
    (u"samskrita", "sanskrit"),
    (u"ह8नदी", "hindi"),
    (u"हिन्दि", "hindi"),
    (u"हिन्दी", "hindi"),
    (u"हिंदी", "hindi"),
    (u"चीनी", "chinese"),
    (u"maardwari", "marwari"),
    (u"maarwadi", "marwari"),
    (u"marwadi", "marwari"),
    (u"मारवाड़ी", "marwari"),
    (u"bangla", "bengali"),
    (u"espagnol", "spanish"),
    (u"sambalpuria", "sambalpuri"),
    (u"samskrutham", "sanskrit"),
    (u"angrezi", "english"),
    (u"gujrathi", "gujarati"),
    (u"gujrati", "gujarati"),
    (u"hindhi", "hindi"),
    (u"hinfi", "hindi"),
    (u"तेलुगू।", "telugu"),
    (u"తెలుగు", "telugu"),
    (u"punjabi", "panjabi"),
    (u"पंजाबी", "panjabi"),
    (u"फ्रेंच", "french"),
    (u"भोजपुरी", "bhojpuri"),
    (u"असमिया", "assamese"),
    (u"देवनागरी", "devanagari"),
    (u"नेपाल", "nepali"),
    (u"नेवारी", "newari"),
    (u"बंगाली", "bengali"),
    (u"बांग्ला", "bengali"),
    (u"башкирский", "bashkir"),
    (u"вьетнамский", "vietnamese"),
    (u"греческий", "greek"),
    (u"greek", "greek"),
    (u"китайский", "chinese"),
    (u"рюске", "russian"),
    (u"татарский", "tatar"),
    (u"финский", "finnish"),
    (u"якутский", "yakut"),
    (u"थमिल", "tamil"),
    (u"грузинский", "georgian"),
    (u"нидерландски", "dutch"),
    (u"odia", "oriya"),
    (u"kutchi", "kachchi"),

])
# sort from longest to the shortest
LANGS_CORRECTIONS_SEARCH_KEYS = list(reversed(sorted(LANGS_CORRECTIONS_SEARCH.keys(), key=len)))

# initial names that are parts of final names should go here
LANGS_CORRECTIONS_EXACT = {
    # "german-german-swiss": "german-swiss", # fix
    # "german-swissgerman": "german-swiss", # fix
    "swiss": "german-swiss",
    # "chinese-chinese-mandarin": "chinese-mandarin", # fix
    "mandarin": "chinese-mandarin",
    # "indonesiann": "indonesian", # fix
    "indonesia": "indonesian",
    "greek": "greek-modern",
    "hind": "hindi",
    "ukr": "ukrainian",
    "ang": "english",
    "de": "german",
    "en": "english",
    "eng": "english",
    "cs": "czech",
    "cz": "czech",
    "fr": "french",
    "ru": "russian",
    u"рб": "belarusian",
    "-": "",
}


def normalise_languages(langs_series):
    """
    Cleanup and normalise languages input from the forms.
    """

    langs_series_out = []
    for s in langs_series:
        s = s.lower()
        # apply search & replace corrections
        for o in LANGS_CORRECTIONS_SEARCH_KEYS:
            s = s.replace(o, LANGS_CORRECTIONS_SEARCH[o])
        # remove punctuation
        for x in PUNCTUATION:
            s = s.replace(x, ",")
        # s = s.replace(u" и ", ",").replace(u" and ", ",").replace("&", ",").replace("/", ",").replace(" ", ",").replace(".", ",")

        langs = []
        for l in s.split(","):
            ln = l.strip()
            # apply exact corrections
            if ln in LANGS_CORRECTIONS_EXACT:
                ln = LANGS_CORRECTIONS_EXACT[ln]
            if ln and ln != "" and ln != "none" and ln not in ANDS:
                langs.append(ln)

        if langs != []:
            langs_series_out.append(",".join(list(set(langs))))
        else:
            langs_series_out.append("none")
    return Series(langs_series_out)


# ----------------------------------------------------------------------
# For checking language names against ISO-639-3

def import_iso_639_3():
    """
    Import ISO 639-3 languages
    """

    isopath = os.path.join("raw_data", "iso-639-3.xml")
    isotree = ET.parse(isopath)
    language_names = []
    for lang in isotree.getroot():
        language_names.append(lang.attrib["name"])
    return language_names


EXPECTED_LANGUAGES = [ln.lower() for ln in import_iso_639_3()] + ["chinese-cantonese", "chinese-mandarin", "chinese-simplified", "german-swiss", "greek-modern", "greek-classical", "portuguese-brasilian", "scottish-gaelic", "spanish-mexican", "bahasa", "creole", "frisian", "hokkien", "devanagari", "gurmukhi", "none"]  # adding some exceptions


# ----------------------------------------------------------------------
# Mapping languages to corresponding scripts

SCRIPT_LANGUAGES = {
    "latin": ["german", "galician", "polish", "icelandic", "turkish", "german-swiss", "hungarian", "catalan", "finnish", "serbian", "croatian", "portuguese", "czech", "estonian", "dutch", "faroese", "creole", "latin", "portuguese-brasilian", "french", "afrikaans", "uzbek", "vietnamese", "italian", "tatar", "slovak", "spanish", "spanish-mexican", "danish", "scottish-gaelic", "irish", "swedish", "slovenian", "latvian", "welsh", "lithuanian", "norwegian", "english", "frisian", "indonesian", "bahasa", "bosnian", "malay"],
    "cyrillic": ["belarusian", "yakut", "macedonian", "serbian", "bashkir", "uzbek", "tatar", "bulgarian", "ukrainian", "kazakh", "bosnian", "russian"],
    "devanagari": ["sanskrit", "devanagari", "hindi", "konkani", "sindhi", "rajasthani", "marathi", "newari", "marwari", "maithili", "bhojpuri", "garhwali", "kashmiri", "nepali"],
    "other": ["urdu", "persian", "arabic", "chinese-cantonese", "hokkien", "chinese", "chinese-mandarin", "chinese-simplified", "greek-classical", "greek-modern", "kachchi", "gujarati", "oriya", "sambalpuri", "telugu", "panjabi", "gurmukhi", "bengali", "assamese", "japanese", "javanese", "armenian", "georgian", "tamil", "none", "tagalog", "malayalam", "kannada", "syriac", "hebrew", "korean"],
}

In [79]:
# DataFrame columns:
CONTROL_COLUMNS = ["date", "native languages", "fluent languages", "age", "reading skills", "design skills",
                   "order", "typeface", "script"]
IGNORE_COLUMNS = ["ip_address", "id", "form_id", "spam", "unnamed: 0", "email", "unnamed: 0.1"]
SCRIPT_TAG = {"devanagari": "deva", "cyrillic": "cyrl", "latin": "latn"}


def readTriplets(path):
    """
    Read triplets and responses from CSV & XLSX exported
    from GetForm and FormKeep (manually editted).
    """

    if not os.path.exists(path):
        logging.error("File does not exist")
    if os.path.isfile(path):
        logging.info("Reading data from: %s" % path)
        if path.endswith("csv"):
            data = pd.read_csv(path, encoding="utf-8")
        elif path.endswith("xls") or path.endswith("xlsx"):
            data = pd.read_excel(path)
        else:
            return None

        # make multiindex for columns and fix small things
        columns = []
        for c in data.columns:
            c = c.lower()
            if c in IGNORE_COLUMNS:
                columns.append(("ignore", "ignore", c))
            elif c in CONTROL_COLUMNS:
                columns.append(("control", "control", c))
            elif c.startswith("order"):
                columns.append(("ignore", "ignore", c))
            else:
                # fix column names for Cyrillic characters
                c = c.replace("short-i", "short i").replace("hard-sign", "hard sign").replace("soft-sign", "soft sign").lower()
                typeface, script, triplet = c.split("_")
                # normalize typeface names
                typeface = typeface.replace(" ", "-").lower()
                script = script.lower()
                chars = triplet.split("-")
                if script == "devanagari":
                    # capitalize devanagari character names
                    chars = [SCRIPT_TAG[script]+"."+char.capitalize() for char in chars]
                else:
                    # remove spaces from two-word character names
                    chars = [SCRIPT_TAG[script]+"."+char.replace(" ", "-") for char in chars]
                triplet = str(sorted(chars))
                columns.append((script, typeface, triplet))
        columns = pd.MultiIndex.from_tuples(columns, names=["script", "typeface", "triplet"])
        data.columns = columns
        data["control", "control", "order"] = ""
        data["control", "control", "script"] = ""
        data["control", "control", "typeface"] = ""

        for i, _ in data.iterrows():
            for c in data.loc[i].index:
                if c[0] not in ["control", "ignore"] and c[2] not in ["control", "ignore"] and pd.notnull(data.loc[i][c]):
                    script = c[0]
                    typeface = c[1]
                    data.at[i, ("control", "control", "script")] = script
                    data.at[i, ("control", "control", "typeface")] = typeface
            for c in data.loc[i].index:
                if c[2] in IGNORE_COLUMNS or c[2] in CONTROL_COLUMNS:
                    pass
                elif c[2].startswith("order") and pd.notnull(data.loc[i][c]):
                    data.at[i, ("control", "control", "order")] = int(data.loc[i][c])
                elif pd.notnull(data.loc[i][c]):
                    # prefix response
                    if script == "devanagari":
                        # capitalize devanagari character names
                        data.at[i, c] = SCRIPT_TAG[script] + "." + data.loc[i][c].capitalize()
                    else:
                        data.at[i, c] = SCRIPT_TAG[script] + "." + data.loc[i][c]
        logging.info("  -> importing %s values" % len(data.index))
        del data["ignore"]
        return data

In [73]:
# read triplets from all CSVs in the source folder
data_all = []
for path_pattern in LOAD_FROM:
    for path in glob.glob(path_pattern):
        data_all.append(readTriplets(path))
data_all = pd.concat(data_all)
data_all["control", "control", "native in script"] = False
data_all["control", "control", "fluent in script"] = False
data_all.reset_index(drop=True, inplace=True)
data_all.index.name = "id"

Reading data from: raw_data/devanagari/itf-devanagari.xls
  -> importing 36 values
Reading data from: raw_data/devanagari/lohit.xls
  -> importing 63 values
Reading data from: raw_data/devanagari/kohinoor.xls
  -> importing 36 values
Reading data from: raw_data/devanagari/ek-mukta.xls
  -> importing 58 values
Reading data from: raw_data/devanagari/devanagari-mt.xls
  -> importing 55 values
Reading data from: raw_data/devanagari/adobe-devanagari.xls
  -> importing 50 values
Reading data from: raw_data/devanagari/murty-hindi.xls
  -> importing 62 values
Reading data from: raw_data/devanagari/nirmala-ui.xls
  -> importing 51 values
Reading data from: raw_data/devanagari/_devanagari_initial.xlsx
  -> importing 42 values
Reading data from: raw_data/cyrillic/arial.xls
  -> importing 50 values
Reading data from: raw_data/cyrillic/courier-new.xls
  -> importing 44 values
Reading data from: raw_data/cyrillic/verdana.xls
  -> importing 45 values
Reading data from: raw_data/cyrillic/georgia.xls
 

In [ ]:
# make a copy
data = pd.DataFrame(data_all)

# normalize language names

data["control", "control", "native languages"] = normalise_languages(data["control", "control", "native languages"].astype("unicode"))
data["control", "control", "fluent languages"] = normalise_languages(data["control", "control", "fluent languages"].astype("unicode"))

# resolve script fluency and nativity

for i, row in data.iterrows():
    script = row["control", "control", "script"]
    # set True/False if native/fluent in the script of the study
    native_in_script = False
    fluent_in_script = False
    for lang in row["control", "control", "native languages"].split(","):
        if lang in SCRIPT_LANGUAGES[script]:
            native_in_script = True
            fluent_in_script = True  # if native then also fluent
    for lang in row["control", "control", "fluent languages"].split(","):
        if lang in SCRIPT_LANGUAGES[script]:
            fluent_in_script = True
    # set True/False if native/fluent in the script of the study
    data.loc[i, ("control", "control", "native in script")] = native_in_script
    data.loc[i, ("control", "control", "fluent in script")] = fluent_in_script

# fix values in reading skills
data["control", "control", "reading skills"] = data["control", "control", "reading skills"].str.replace("_", " ")
# fix values in age
# data["control", "control", "age"] = data["control", "control", "age"].str.replace("-", "-") # en dash -> hyphen
# remove illiterate participants
data = data[(data["control", "control", "fluent languages"] != "none") | (data["control", "control", "native languages"] != "none")]

# reorder columns
cc = [col for col in data.columns if col[0] == "control"]
ncc = [col for col in data.columns if col[0] != "control"]
data = pd.DataFrame(data, columns=cc+ncc, index=data.index)
data.sort_index(axis=1, inplace=True)

# sort by date
data.sort_values([("control", "control", "date")], axis=0, inplace=True)
data.reset_index(drop=True, inplace=True)

# save all to one file
path = os.path.join(SAVE_TO, "raw-data-preprocessed.csv")
data.to_csv(path, encoding="utf-8")
logging.info("Saving all data to: %s" % path)
logging.info("Saving the data to individual files.")
logging.info("Statistics for typefaces and script (all/non-designers):")

logging.info("Languages (new, unexpected languages):")
# get a list of all languages
all_langs = list(data["control", "control", "native languages"])+list(data["control", "control", "fluent languages"])
languages = list(set((",".join(all_langs)).split(",")))
# print unexpected languages
for lang in sorted(languages):
    if lang not in EXPECTED_LANGUAGES:
        print(lang)  # the printout looks nicer this way
else:
    print("—")


Saving all data to: csv/raw-data-preprocessed.csv
Saving the data to individual files.
Statistics for typefaces and script (all/non-designers):
Languages (new, unexpected languages):


chinese-chinese-chinese-chinese-chinese-chinese-chinese-chinese-chinese-chinese-chinese-cantonese
—
